<a href="https://colab.research.google.com/github/EAsencios/DEEP-LEARING/blob/master/fi_on_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky es very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'
]
labels = ['weather', 'weather', 'animal', 'animal', 'weather', 'animal']
corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document':corpus,
                          'Category':labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animal
3,The brown fox is quick and the blue dog is lazy!,animal
4,The sky es very blue and the sky is very beaut...,weather
5,The dog is lazy but the brown fox is quick!,animal


In [3]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

In [11]:
def normalize_document(doc):
  # lower case and remove special characters\whitespaces
  doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
  doc = doc.lower()
  doc = doc.strip()
  # tokenize document
  tokens = wpt.tokenize(doc)
  # filter stopwords out of document
  filtered_tokens = [token for token in tokens if token not in stop_words]
  # re-create document from filtered tokens
  doc = ' '.join(filtered_tokens)
  return doc

In [12]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog', 'brown fox quick blue dog lazy',
       'sky es blue sky beautiful today', 'dog lazy brown fox quick'],
      dtype='<U31')

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0, max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1],
       [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]])

In [15]:
vocab = cv.get_feature_names()
pd.DataFrame(cv_matrix, columns=vocab)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beautiful,blue,brown,dog,es,fox,jumps,lazy,love,quick,sky,today
0,1,1,0,0,0,0,0,0,0,0,1,0
1,1,1,0,0,0,0,0,0,1,0,1,0
2,0,0,1,1,0,1,1,1,0,1,0,0
3,0,1,1,1,0,1,0,1,0,1,0,0
4,1,1,0,0,1,0,0,0,0,0,2,1
5,0,0,1,1,0,1,0,1,0,1,0,0


In [17]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)
bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beautiful sky,beautiful today,blue beautiful,blue dog,blue sky,brown fox,dog lazy,es blue,fox jumps,fox quick,jumps lazy,lazy brown,lazy dog,love blue,quick blue,quick brown,sky beautiful,sky blue,sky es
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0
3,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0
4,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1
5,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0, max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.around(tv_matrix, 2), columns=vocab)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beautiful,blue,brown,dog,es,fox,jumps,lazy,love,quick,sky,today
0,0.60,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00
1,0.46,0.39,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.00,0.46,0.00
2,0.00,0.00,0.38,0.38,0.00,0.38,0.54,0.38,0.00,0.38,0.00,0.00
3,0.00,0.36,0.42,0.42,0.00,0.42,0.00,0.42,0.00,0.42,0.00,0.00
4,0.32,0.27,0.00,0.00,0.46,0.00,0.00,0.00,0.00,0.00,0.64,0.46
5,0.00,0.00,0.45,0.45,0.00,0.45,0.00,0.45,0.00,0.45,0.00,0.00


In [21]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5
0,1.000000,0.753128,0.000000,0.185447,0.717486,0.000000
1,0.753128,1.000000,0.000000,0.139665,0.540359,0.000000
2,0.000000,0.000000,1.000000,0.784362,0.000000,0.839987
3,0.185447,0.139665,0.784362,1.000000,0.097425,0.933779
4,0.717486,0.540359,0.000000,0.097425,1.000000,0.000000
5,0.000000,0.000000,0.839987,0.933779,0.000000,1.000000
